In [1]:
import sys
import pygame
from pygame.locals import *
import pymunk 

from pymunk.vec2d import Vec2d

pygame 1.9.6
Hello from the pygame community. https://www.pygame.org/contribute.html
Loading chipmunk for Darwin (64bit) [/Users/alecx/anaconda3/envs/physics/lib/python2.7/site-packages/pymunk/libchipmunk.dylib]


In [2]:
from pymunk import pygame_util
import random
import math

In [3]:
def add_bullet(space):
    mass = 1
    radius = 5
    moment = pymunk.moment_for_circle(mass, 0, radius) # 1
    body = pymunk.Body(mass, moment) # 2
    body.position = 0, 200 # 3
    body.start_position = Vec2d(body.position)
    shape = pymunk.Circle(body, radius) # 4
    
    shape.elasticity = 0
    shape.friction = 1000000
    shape.collision_type = 1
    
    shape.color = pygame.color.THECOLORS["black"]
    
    space.add(body, shape) # 5
    return shape

In [4]:
def add_pendulum(space):
    
    # x and y coords for block's center
    center_x = 300
    center_y = 100
    
    # Add pendulum's block
    #block_mass = 10000    
    #block_moment = pymunk.moment_for_box(block_mass, (200,100))
    block_body = pymunk.Body()
    
    block_body.position = center_x, center_y
    block_shape = pymunk.Poly(block_body, [(center_x-100, center_y-50),(center_x-100, center_y+50),(center_x+100, center_y+50),(center_x+100, center_y-50)])
    block_shape.density = .0025
    
    block_shape.elasticity = 0
    block_shape.friction = 10000000
    block_shape.collision_type = 2
    
    block_shape.color = pygame.color.THECOLORS["gray"]
    
    space.add(block_body, block_shape)    
    
    
    # Make pendulum hang from two strings
    pivot_point_1 = pymunk.Body(body_type = pymunk.Body.STATIC)
    pivot_point_1.position = (center_x, center_y+100)
    
    pivot_point_2 = pymunk.Body(body_type = pymunk.Body.STATIC)
    pivot_point_2.position = (center_x, center_y+100)
    
    joint_1 = pymunk.constraint.PinJoint(pivot_point_1, block_body, (center_x-100, center_y+100), (center_x-100,center_y+50))
    joint_2 = pymunk.constraint.PinJoint(pivot_point_2, block_body, (center_x+100, center_y+100), (center_x+100,center_y+50))
    
    space.add(joint_1, joint_2)
    
    return block_shape

In [5]:
def bullet_hits(arbiter, space, _):    
    '''
    This function is called after the bullet finishes transferring momentum to block (during post_solve)
    '''
    # Transfer mass of bullet to block
    space.bodies[1].mass += (space.bodies[0].mass)
    
    space.remove(space.shapes[0]) # remove the bullet shape
    space.remove(space.bodies[0]) # remove the bullet body


In [7]:
def main():
    pygame.init()
    myfont = pygame.font.SysFont('Comic Sans MS', 30)
    
    screen_width, screen_height = 800, 600
    screen = pygame.display.set_mode((screen_width, screen_height))
    pygame.display.set_caption("Ballistic Pendulum Simulation")
    clock = pygame.time.Clock()

    background = pygame.Surface(screen.get_size())
    background = background.convert()
    background.fill((255, 255, 255))
        
    
    space = pymunk.Space() #2
    space.gravity = 0, -900
    space.damping = .99
    
    bullet = add_bullet(space)
    block = add_pendulum(space)
    
    # Setup bullet-block collision callback function
    h = space.add_collision_handler(1, 2) # ...(COLLTYPE_BULLET, COLLTYPE_BLOCK)
    h.post_solve = bullet_hits
    

    draw_options = pymunk.pygame_util.DrawOptions(screen)
    draw_options.constraint_color = pygame.color.THECOLORS["brown"]
    draw_options.collision_point_color = pygame.color.THECOLORS["red"] 


    space.shapes[0].body.apply_impulse_at_local_point((15000,0))
    
    
    
    max_height_reached = None
    max_potential_energy = None
    
    while True:
        for event in pygame.event.get():
            if event.type == QUIT:
                sys.exit(0)
            elif event.type == KEYDOWN and event.key == K_ESCAPE:
                sys.exit(0)
                
                
        screen.blit(background, (0, 0))
        
        # Ongoing Calculations
        text_h = myfont.render("Block Height: %d" % (block.body.position[1] - 100), True, (0, 0, 0), (255,255,255))
        screen.blit(text_h, (screen_width/2-text_h.get_rect().width/2, screen_height/2 - 200))
        
        text_ke = myfont.render("Block KE: %d" % (block.body.kinetic_energy), True, (0, 0, 0), (255,255,255))
        screen.blit(text_ke, (screen_width/2-text_ke.get_rect().width/2, screen_height/2 - 250))
        
        # One-time Calculations (when pendulum reaches max-height)
        if(block.body.kinetic_energy < 30):
            max_height_reached = block.body.position[1] - 100
            max_potential_energy = block.body.mass*900*(block.body.position[1] - 100)
            
            
        if(max_height_reached != None):
            text_maxh = myfont.render("Max Height Reached: %d" % max_height_reached, True, (0, 0, 0), (255,255,255))
            screen.blit(text_maxh, (screen_width/2-text_maxh.get_rect().width/2, screen_height/2 + 200))

            text_mgh = myfont.render("Max PE Reached: %d" % max_potential_energy, True, (0, 0, 0), (255,255,255))
            screen.blit(text_mgh, (screen_width/2-text_mgh.get_rect().width/2, screen_height/2 + 250))
                
        space.step(1/500.0) # TEMP: Slow-motion
        
        
        
        
        space.debug_draw(draw_options)


        pygame.display.flip()
        clock.tick(50)
        

if __name__ == '__main__':
    sys.exit(main())

SystemExit: 0

/Users/alecx/anaconda3/envs/physics/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2886: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
